In [1]:
import sys
sys.path.append('..')

import pandas as pd
from src.models.baseline import ExtractiveSummarizer
from src.models.transformer import TransformerSummarizer
import warnings
warnings.filterwarnings('ignore')

c:\Users\ASUS\anaconda3\envs\med_summery_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_df = pd.read_csv('../data/processed/test.csv')
print(f"Test set size: {len(test_df)}")
test_df.head()

Test set size: 424


,report,summary
0,"VITRECTOMY OPENING,The patient was brought to ...",Vitrectomy opening. A limited conjunctival per...
1,"EXAM:,1. Diagnostic cerebral angiogram.,2. Tra...",Diagnostic cerebral angiogram and transcathete...
2,"CC:, Rapidly progressive amnesia.,HX: ,This 63...",Patient with a 1-year history of progressive a...
3,"CHIEF COMPLAINT:, Followup on diabetes mellitu...",Sample progress note - Gen Med.
4,"SUBJECTIVE:, The patient is a 75-year-old fema...",A 75-year-old female comes in with concerns of...


In [3]:
extractive_summarizer = ExtractiveSummarizer(num_sentences=2)

In [4]:
sample_idx = 0
sample_report = test_df.iloc[sample_idx]['report']
sample_summary = test_df.iloc[sample_idx]['summary']

print("="*80)
print("ORIGINAL REPORT:")
print("="*80)
print(sample_report)
print("\n" + "="*80)
print("REFERENCE SUMMARY:")
print("="*80)
print(sample_summary)
print("\n" + "="*80)
print("EXTRACTIVE SUMMARY:")
print("="*80)
extractive_summary = extractive_summarizer.summarize(sample_report)
print(extractive_summary)

ORIGINAL REPORT:
VITRECTOMY OPENING,The patient was brought to the operating room and appropriately identified. General anesthesia was induced by the anesthesiologist. The patient was prepped and draped in the usual sterile fashion. A lid speculum was used to provide exposure to the right eye. A limited conjunctival peritomy was created with Westcott scissors to expose the supranasal and separately the supratemporal and inferotemporal quadrants. Hemostasis was maintained with wet-field cautery. Calipers were set at XX mm and the mark was made XX mm posterior to the limbus in the inferotemporal quadrant. A 5-0 nylon suture was passed through partial-thickness sclera on either side of this mark. The MVR blade was used to make a sclerotomy between the preplaced sutures. An 8-0 nylon suture was then preplaced for a later sclerotomy closure. The infusion cannula was inspected and found to be in good working order. The infusion cannula was placed into the vitreous cavity and secured with the

In [5]:
print("Loading T5 model...")
t5_summarizer = TransformerSummarizer("t5-small")

# Cell 5
# Test T5 on the same sample
print("="*80)
print("T5 GENERATED SUMMARY:")
print("="*80)
t5_summary = t5_summarizer.summarize(sample_report, max_length=128)
print(t5_summary)

Loading T5 model...
Loading model: t5-small
Using device: cuda
T5 GENERATED SUMMARY:
anesthesia was induced by the anesthesiologist. the patient was prepped and draped in the usual sterile fashion. a limited conjunctival peritomy was created with westcott scissors.


In [6]:
num_samples = 3

for i in range(num_samples):
    print(f"\n{'='*80}")
    print(f"SAMPLE {i+1}")
    print(f"{'='*80}")
    
    report = test_df.iloc[i]['report']
    reference = test_df.iloc[i]['summary']
    
    extractive = extractive_summarizer.summarize(report)
    t5_gen = t5_summarizer.summarize(report, max_length=128)
    
    print(f"\nREFERENCE:\n{reference}")
    print(f"\nEXTRACTIVE:\n{extractive}")
    print(f"\nT5 (pretrained):\n{t5_gen}")


SAMPLE 1

REFERENCE:
Vitrectomy opening. A limited conjunctival peritomy was created with Westcott scissors to expose the supranasal and separately the supratemporal and inferotemporal quadrants.

EXTRACTIVE:
The infusion cannula was placed into the vitreous cavity and secured with the preplaced suture. The tip of the infusion cannula was directly visualized and found to be free of any overlying tissue and the infusion was turned on.

T5 (pretrained):
anesthesia was induced by the anesthesiologist. the patient was prepped and draped in the usual sterile fashion. a limited conjunctival peritomy was created with westcott scissors.

SAMPLE 2

REFERENCE:
Diagnostic cerebral angiogram and transcatheter infusion of papaverine

EXTRACTIVE:
The images showed spasm of the left internal carotid artery and the left A1, it was thought planned to infused papaverine into the ICA and the left A1. The microcatheter was then taken up into the internal carotid artery under biplane roadmapping and was t